In [1]:
import sys
import json
import requests
import re
import numpy as np
import pandas as pd
import pymysql as pm
# 쿼리문 그대로 박아서 쓸 수 있음 데이터 읽어들일 때 사용 
from sqlalchemy import create_engine
# 데이터프레임 단위로 mysql db에 적재 가능, 만든거 db에 박을때 사용 가능 
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import re
import math as m
from dateutil.relativedelta import relativedelta
from google.cloud import bigquery

/Users/gowid/Library/Python/3.8/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
client_gowid_gcs_dev = bigquery.Client(project='gowid-gcs-dev')

In [3]:

if __name__ == '__main__':
    sql_path = './sql/'
    
    def sql_card(_file_name):
        with open(str(sql_path)+_file_name, 'r') as q:
            sql = q.read()
            return client_gowid_gcs_dev.query(sql)

In [4]:
# pg_tracker base table 가져오기
base = sql_card('pg_tracker.sql').to_dataframe()

In [8]:
#dictionary 가공
# 불러오고 include / exclude 나누기
pg_dic = sql_card('dic_pg_tracker.sql').to_dataframe()
pg_dic_in = pg_dic.loc[pg_dic['pg_category'] == 'Include']
pg_dic_ex = pg_dic.loc[pg_dic['pg_category'] == 'Exclude']
# include -> str 처리
pg_dic_in_str = pg_dic_in[['pg_desc']].to_string(header=False, index = False, index_names = False).split('\n')
pg_dic_in_str = ",".join(pg_dic_in_str).replace(",","|").replace(" ","")
# exclude -> str 파라미터 처리
pg_dic_ex_str = pg_dic_ex[['pg_desc']].to_string(header=False, index = False, index_names = False).split('\n')
pg_dic_ex_str = ",".join(pg_dic_ex_str).replace(",","|").replace(" ","")

pg_dic_in_str

'코레일|나이스페이먼츠|티몬|온오프코리아|이삭랜드|교보증권|다우데이타|원페이|페이업|코엠페이먼츠|미래에셋증권|롯데멤버스|한화투자증권|주식회사모인|스마트로|케이지모빌리언스|페이레터|핀샷|원스토어|주식회사더즌|비전씨|루멘페이먼츠|핀플넷|인스타페이|아이씨비|토스페이먼츠|이후에프엔비|다모페이|스탠다드네트웍스|페이스토리|구글코리아|핀크|케이원피에스|로드시스템|코페이|주다날|티엔디엔|십일번가|뱅크샐러드|페이누리|KCP|센트비|결제선생|롯데쇼핑|다모아페이|KG이니시스|CJ올리브네트웍스|하렉스인포텍|위메프|시루정보|나이스정보통신|로카모빌리티|웰컴|원넷|네이버파이낸셜|야놀자|페이게이트|아고다페이먼트|한페이시스|블루월넛|카카오페이|비즈플레이|이나인페이|이지스엔터프라이즈|에스케이플래닛|한국철도|디셈버앤|효성에프엠에스|티모넷|유디아이디|오렌지스퀘어|트래블파트너|한국투자증권|이투유|tosspaymen|엔에이치엔페이|페이네스트|더존비즈온|E-메리츠|쿠팡|섹타나인|에프케이|스토리정보통신|아이오로라|유핀테크|우아한형제들|페이플|한국조폐|페이봇|스마트온라인|당근페이|주핑거|한패스|커넥|코밴|오케이인베스트|유렉스|사이렉스페이|SK증권|주식회사컬리|씨제이올리브네트웍스|파이서브코리아|제이티넷|유안타증권|13마일|코리아결제|패스고|아이에스이커머스|유진투자증권|웰컴페이먼츠|차이코퍼레이션|더페이|주카카오|비바리퍼블리카|윈글로벌페이|더페이원|한국결제인증|세븐페이|티머니|갤럭시아머니트리|유니윌지점위즈|지오엠아이앤|이롬넷|블루월넛|주스마트로|케이아이에스|에스에스지닷컴|제로페이|한국문화진흥|엔에이치투자|한국신용카드결제|통통|Toss정산|마이비|동서아이티에스|신한금융|삼성증권|없음|인피니소프트|부산하나로카드|디지비유페이|주식회사케이티|위대한상상|지머니트랜스|트래블월렛|카카오모빌리티|와이어바알리|케이에스넷|한국정보통신|카페24주식회사|주케이티|결제PG|인터파크|엘지씨엔에스|엔에이치엔|동부증권|현대차증권|피앤링크|골프존|부국위너스|금융결제원|옵타움플랫폼|지엘엔인터내셔널|에스엠하이플러스|에스케이커뮤니케이션|케이비증

In [128]:
# pg 매출과 일반매출 분류
base['pg_flag'] = (base['resaccount_final'].str.contains(pg_dic_in_str) == True) & (base['resaccount_final'].str.contains(pg_dic_ex_str) == False)
base.groupby(['pg_flag'])['pg_flag'].count()

pg_flag
False    4064009
True      415242
Name: pg_flag, dtype: int64